Define a board for Tic-Tac-Toe

In [93]:
import numpy as np
from enum import Enum


class GameResult(Enum):
    """
    Enum to encode different states of the game. A game can be in progress (NOT_FINISHED), lost, won, or draw
    """
    NOT_FINISHED = 0
    NAUGHT_WIN = 1
    CROSS_WIN = 2
    DRAW = 3


#
# Values to encode the current content of a field on the board. A field can be empty, contain a naught, or
# contain a cross
#
EMPTY = 0  # type: int
NAUGHT = 1  # type: int
CROSS = 2  # type: int

#
# Define the length and width of the board. Has to be 3 at the moment, or some parts of the code will break. Also,
# the game mechanics kind of require this dimension unless other rules are changed as well. Encoding as a variable
# to make the code more readable
#
BOARD_DIM = 3  # type: int
BOARD_SIZE = BOARD_DIM * BOARD_DIM  # type: int


class Board:
    """
    The class to encode a tic-tac-toe board, including its current state of pieces.
    Also contains various utility methods.
    """

    #
    # We will use these starting positions and directions when checking if a move resulted in the game being
    # won by one of the sides.
    #
    WIN_CHECK_DIRS = {0: [(1, 1), (1, 0), (0, 1)],
                      1: [(1, 0)],
                      2: [(1, 0), (1, -1)],
                      3: [(0, 1)],
                      6: [(0, 1)]}

    def hash_value(self) -> int:
        """
        Encode the current state of the game (board positions) as an integer. Will be used for caching evaluations
        :return: A collision free hash value representing the current board state
        """
        res = 0
        for i in range(BOARD_SIZE):
            res *= 3
            res += self.state[i]

        return res

    @staticmethod
    def other_side(side: int) -> int:
        """
        Utility method to return the value of the other player than the one passed as input
        :param side: The side we want to know the opposite of
        :return: The opposite side to the one passed as input
        """
        if side == EMPTY:
            raise ValueError("EMPTY has no 'other side'")

        if side == CROSS:
            return NAUGHT

        if side == NAUGHT:
            return CROSS

        raise ValueError("{} is not a valid side".format(side))

    def __init__(self, s=None):
        """
        Create a new Board. If a state is passed in, we use that otherwise we initialize with an empty board
        :param s: Optional board state to initialise the board with
        """
        if s is None:
            self.state = np.ndarray(shape=(1, BOARD_SIZE), dtype=int)[0]
            self.reset()
        else:
            self.state = s.copy()

    def coord_to_pos(self, coord: (int, int)) -> int:
        """
        Converts a 2D board position to a 1D board position.
        Various parts of code prefer one over the other.
        :param coord: A board position in 2D coordinates
        :return: The same board position in 1D coordinates
        """
        return coord[0] * BOARD_DIM + coord[1]

    def pos_to_coord(self, pos: int) -> (int, int):
        """
        Converts a 1D board position to a 2D board position.
        Various parts of code prefer one over the other.
        :param pos: A board position in 1D coordinates
        :return: The same board position in 2D coordinates
        """
        return pos // BOARD_DIM, pos % BOARD_DIM

    def reset(self):
        """
        Resets the game board. All fields are set to be EMPTY.
        """
        self.state.fill(EMPTY)

    def num_empty(self) -> int:
        """
        Counts and returns the number of empty fields on the board.
        :return: The number of empty fields on the board
        """
        return np.count_nonzero(self.state == EMPTY)

    def random_empty_spot(self) -> int:
        """
        Returns a random empty spot on the board in 1D coordinates
        :return: A random empty spot on the board in 1D coordinates
        """
        index = np.random.randint(self.num_empty())
        for i in range(9):
            if self.state[i] == EMPTY:
                if index == 0:
                    return i
                else:
                    index = index - 1

    def is_legal(self, pos: int) -> bool:
        """
        Tests whether a board position can be played, i.e. is currently empty
        :param pos: The board position in 1D that is to be checked
        :return: Whether the position can be played
        """
        return (0 <= pos < BOARD_SIZE) and (self.state[pos] == EMPTY)

    def move(self, position: int, side: int) -> (np.ndarray, GameResult, bool):
        """
        Places a piece of side "side" at position "position". The position is to be provided as 1D.
        Throws a ValueError if the position is not EMPTY
        returns the new state of the board, the game result after this move, and whether this move has finished the game
        :param position: The position where we want to put a piece
        :param side: What piece we want to play (NAUGHT, or CROSS)
        :return: The game state after the move, The game result after the move, Whether the move finished the game
        """
        if self.state[position] != EMPTY:
            print('Illegal move')
            raise ValueError("Invalid move")

        self.state[position] = side

        if self.check_win():
            return self.state, GameResult.CROSS_WIN if side == CROSS else GameResult.NAUGHT_WIN, True

        if self.num_empty() == 0:
            return self.state, GameResult.DRAW, True

        return self.state, GameResult.NOT_FINISHED, False

    def apply_dir(self, pos: int, direction: (int, int)) -> int:
        """
        Applies 2D direction dir to 1D position pos.
        Returns the resulting 1D position, or -1 if the resulting position would not be a valid board position.
        Used internally to check whether either side has won the game.
        :param pos: What position in 1D to apply the direction to
        :param direction: The direction to apply in 2D
        :return: The resulting 1D position, or -1 if the resulting position would not be a valid board position.
        """
        row = pos // 3
        col = pos % 3
        row += direction[0]
        if row < 0 or row > 2:
            return -1
        col += direction[1]
        if col < 0 or col > 2:
            return -1

        return row * 3 + col

    def check_win_in_dir(self, pos: int, direction: (int, int)) -> bool:
        """
        Checks and returns whether there are 3 pieces of the same side in a row if following direction dir
        Used internally to check whether either side has won the game.
        :param pos: The position in 1D from which to check if we have 3 in a row
        :param direction: The direction in 2D in which to check for 3 in a row
        :return: Whether there are 3 in a row of the same side staring from position pos and going in direction
        `direction`
        """
        c = self.state[pos]
        if c == EMPTY:
            return False

        p1 = int(self.apply_dir(pos, direction))
        p2 = int(self.apply_dir(p1, direction))

        if p1 == -1 or p2 == -1:
            return False

        if c == self.state[p1] and c == self.state[p2]:
            return True

        return False

    def who_won(self) -> int:
        """
        Check whether either side has won the game and return the winner
        :return: If one player has won, that player; otherwise EMPTY
        """
        for start_pos in self.WIN_CHECK_DIRS:
            if self.state[start_pos] != EMPTY:
                for direction in self.WIN_CHECK_DIRS[start_pos]:
                    res = self.check_win_in_dir(start_pos, direction)
                    if res:
                        return self.state[start_pos]

        return EMPTY

    def check_win(self) -> bool:
        """
        Check whether either side has won the game
        :return: Whether a side has won the game
        """
        for start_pos in self.WIN_CHECK_DIRS:
            if self.state[start_pos] != EMPTY:
                for direction in self.WIN_CHECK_DIRS[start_pos]:
                    res = self.check_win_in_dir(start_pos, direction)
                    if res:
                        return True

        return False

    def state_to_char(self, pos, html=False):
        """
        Return 'x', 'o', or ' ' depending on what piece is on 1D position pos. Ig `html` is True,
        return '&ensp' instead of ' ' to enforce a white space in the case of HTML output
        :param pos: The position in 1D for which we want a character representation
        :param html: Flag indicating whether we want an ASCII (False) or HTML (True) character
        :return: 'x', 'o', or ' ' depending on what piece is on 1D position pos. Ig `html` is True,
        return '&ensp' instead of ' '
        """
        if (self.state[pos]) == EMPTY:
            return '&ensp;' if html else ' '

        if (self.state[pos]) == NAUGHT:
            return 'o'

        return 'x'

    def html_str(self) -> str:
        """
        Format and return the game state as a HTML table
        :return: The game state as a HTML table string
        """
        data = self.state_to_charlist(True)
        html = '<table border="1"><tr>{}</tr></table>'.format(
            '</tr><tr>'.join(
                '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
        )
        return html

    def state_to_charlist(self, html=False):
        """
        Convert the game state to a list of list of strings (e.g. for creating a HTML table view of it).
        Useful for displaying the current state of the game.
        :param html: Flag indicating whether we want an ASCII (False) or HTML (True) character
        :return: A list of lists of character representing the game state.
        """
        res = []
        for i in range(3):
            line = [self.state_to_char(i * 3, html),
                    self.state_to_char(i * 3 + 1, html),
                    self.state_to_char(i * 3 + 2, html)]
            res.append(line)

        return res

    def __str__(self) -> str:
        """
        Return ASCII representation of the board
        :return: ASCII representation of the board
        """
        board_str = ""
        for i in range(3):
            board_str += self.state_to_char(i * 3) + '|' + self.state_to_char(i * 3 + 1) \
                         + '|' + self.state_to_char(i * 3 + 2) + "\n"

            if i != 2:
                board_str += "-----\n"

        board_str += "\n"
        return board_str

    def print_board(self):
        """
        Print an ASCII representation of the board
        """
        for i in range(3):
            board_str = self.state_to_char(i * 3) + '|' + self.state_to_char(i * 3 + 1) \
                        + '|' + self.state_to_char(i * 3 + 2)

            print(board_str)
            if i != 2:
                print("-----")

        print("")

In [94]:
from IPython.display import HTML, display
def print_board(board):
    display(HTML("""
    <style>
    .rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td {
      border: 1px  black solid !important;
      color: black !important;
    }
    </style>
    """+board.html_str()))
board = Board()
print_board(board)

,,
,,
,,


In [95]:
board.reset()
finished = False
while not finished:
   _, result, finished = board.move(board.random_empty_spot(), CROSS)
   print_board(board)
   if finished:
       if result == GameResult.DRAW:
           print("Game is a draw")
       else:
           print("Cross won!")
   else:
       _, result, finished = board.move(board.random_empty_spot(), NAUGHT)
       print_board(board)
       if finished:
            if result == GameResult.DRAW:
               print("Game is a draw")
            else:
               print("Naught won!")

,,
,,x
,,


,,
,,x
,,o


x,,
,,x
,,o


x,,o
,,x
,,o


x,x,o
,,x
,,o


x,x,o
,,x
,o,o


x,x,o
x,,x
,o,o


x,x,o
x,o,x
,o,o


x,x,o
x,o,x
x,o,o


Cross won!


Define the Player Mode


In [96]:
from abc import ABC, abstractmethod
class Player(ABC):
    """
    Abstract class defining the interface we expect any Tic Tac Toe player class to implement.
    This will allow us to pit various different implementation against each other
    """

    def __init__(self):
        """
        Nothing to do here apart from calling our super class
        """
        super().__init__()

    @abstractmethod
    def move(self, board: Board) -> (GameResult, bool):
       
        pass

    @abstractmethod
    def final_result(self, result: GameResult):
        
        pass

    @abstractmethod
    def new_game(self, side: int):
        
        pass


Define a Default Opponent using a random Minimax

In [97]:
import random


class RndMinMaxAgent(Player):
   

    WIN_VALUE = 1
    DRAW_VALUE = 0
    LOSS_VALUE = -1

    def __init__(self):
        
        self.side = None

        
        self.cache = {}

        super().__init__()

    def new_game(self, side: int):
        
        if self.side != side:
            self.side = side
            self.cache = {}

    def final_result(self, result: GameResult):
       
        pass

    def _min(self, board: Board) -> int:
        

        #
        # First we check if we have seen this board position before, and if yes just return a random choice
        # from the cached values
        #
        board_hash = board.hash_value()
        if board_hash in self.cache:
            return random.choice(self.cache[board_hash])

        #
        # If the game has already finished we return. Otherwise we look at possible continuations
        #
        winner = board.who_won()
        if winner == self.side:
            best_moves = {(self.WIN_VALUE, -1)}
        elif winner == board.other_side(self.side):
            best_moves = {(self.LOSS_VALUE, -1)}
        else:
            #
            # Init the min value as well as action. Min value is set to DRAW as this value will pass through in case
            # of a draw
            #
            min_value = self.DRAW_VALUE
            action = -1
            best_moves = {(min_value, action)}
            for index in [i for i, e in enumerate(board.state) if board.state[i] == EMPTY]:
                b = Board(board.state)
                b.move(index, board.other_side(self.side))

                res, _ = self._max(b)
                if res < min_value or action == -1:
                    min_value = res
                    action = index
                    best_moves = {(min_value, action)}
                elif res == min_value:
                    action = index
                    best_moves.add((min_value, action))

        best_moves = tuple(best_moves)
        self.cache[board_hash] = best_moves

        return random.choice(best_moves)

    def _max(self, board: Board) -> int:
       

        #
        # First we check if we have seen this board position before, and if yes just return a random choice
        # from the cached values
        #
        board_hash = board.hash_value()
        if board_hash in self.cache:
            return random.choice(self.cache[board_hash])

        #
        # If the game has already finished we return. Otherwise we look at possible continuations
        #
        winner = board.who_won()
        if winner == self.side:
            best_moves = {(self.WIN_VALUE, -1)}
        elif winner == board.other_side(self.side):
            best_moves = {(self.LOSS_VALUE, -1)}
        else:
            #
            # Init the min value as well as action. Min value is set to DRAW as this value will pass through in case
            # of a draw
            #
            max_value = self.DRAW_VALUE
            action = -1
            best_moves = {(max_value, action)}
            for index in [i for i, e in enumerate(board.state) if board.state[i] == EMPTY]:
                b = Board(board.state)
                b.move(index, self.side)

                res, _ = self._min(b)
                if res > max_value or action == -1:
                    max_value = res
                    action = index
                    best_moves = {(max_value, action)}
                elif res == max_value:
                    action = index
                    best_moves.add((max_value, action))

        best_moves = tuple(best_moves)
        self.cache[board_hash] = best_moves

        return random.choice(best_moves)

    def move(self, board: Board) -> (GameResult, bool):
       
        score, action = self._max(board)
        _, res, finished = board.move(action, self.side)
        return res, finished

Define Minimax Player

In [98]:
class MinMaxAgent(Player):
    WIN_VALUE = 1
    DRAW_VALUE = 0
    LOSS_VALUE = -1

    def __init__(self):
        self.side = None
        self.cache = {}
        super().__init__()

    def new_game(self, side: int):
        if self.side != side:
            self.side = side
            self.cache = {}

    def final_result(self, result: int):
        pass

    def _min(self, board: Board) -> (float, int):
        board_hash = board.hash_value()
        if board_hash in self.cache:
            return self.cache[board_hash]

        min_value = self.DRAW_VALUE
        action = -1
        winner = board.who_won()

        if winner == self.side:
            min_value = self.WIN_VALUE
        elif winner == board.other_side(self.side):
            min_value = self.LOSS_VALUE
        else:
            for index in [i for i, e in enumerate(board.state) if board.state[i] == EMPTY]:
                b = Board(board.state)
                b.move(index, board.other_side(self.side))
                res, _ = self._max(b)

                if res < min_value or action == -1:
                    min_value = res
                    action = index

                    if min_value == self.LOSS_VALUE:
                        self.cache[board_hash] = (min_value, action)
                        return min_value, action

            self.cache[board_hash] = (min_value, action)

        return min_value, action

    def _max(self, board: Board) -> (float, int):
        board_hash = board.hash_value()
        if board_hash in self.cache:
            return self.cache[board_hash]

        max_value = self.DRAW_VALUE
        action = -1
        winner = board.who_won()

        if winner == self.side:
            max_value = self.WIN_VALUE
        elif winner == board.other_side(self.side):
            max_value = self.LOSS_VALUE
        else:
            for index in [i for i, e in enumerate(board.state) if board.state[i] == EMPTY]:
                b = Board(board.state)
                b.move(index, self.side)
                res, _ = self._min(b)

                if res > max_value or action == -1:
                    max_value = res
                    action = index

                    if max_value == self.WIN_VALUE:
                        self.cache[board_hash] = (max_value, action)
                        return max_value, action

            self.cache[board_hash] = (max_value, action)

        return max_value, action

    def move(self, board: Board) -> (int, bool):
        score, action = self._max(board)
        _, finished = board.move(action, self.side)
        return action, finished


Define Tabluar Q-learning Player

In [99]:
from typing import Dict, List
import numpy as np

WIN_VALUE = 1.0
DRAW_VALUE = 0.5
LOSS_VALUE = 0.0

class TQPlayer(Player):
    def __init__(self, alpha=0.9, gamma=0.95, q_init=0.6):
        self.side = None
        self.q = {}
        self.move_history = []
        self.learning_rate = alpha
        self.value_discount = gamma
        self.q_init_val = q_init
        super().__init__()

    def get_q(self, board_hash: int) -> np.ndarray:
        if board_hash in self.q:
            qvals = self.q[board_hash]
        else:
            qvals = np.full(9, self.q_init_val)
            self.q[board_hash] = qvals
        return qvals

    def get_move(self, board: Board) -> int:
        board_hash = board.hash_value()
        qvals = self.get_q(board_hash)

        while True:
            m = np.argmax(qvals)
            if board.is_legal(m):
                return m
            else:
                qvals[m] = -1.0

    def move(self, board: Board):
        m = self.get_move(board)
        self.move_history.append((board.hash_value(), m))
        _, finished = board.move(m, self.side)
        return m, finished

    def final_result(self, result: GameResult):
        if (result == GameResult.NAUGHT_WIN and self.side == NAUGHT) or (
                result == GameResult.CROSS_WIN and self.side == CROSS):
            final_value = WIN_VALUE
        elif (result == GameResult.NAUGHT_WIN and self.side == CROSS) or (
                result == GameResult.CROSS_WIN and self.side == NAUGHT):
            final_value = LOSS_VALUE
        elif result == GameResult.DRAW:
            final_value = DRAW_VALUE
        else:
            raise ValueError(f"Unexpected game result {result}")

        self.move_history.reverse()
        next_max = -1.0

        for h in self.move_history:
            qvals = self.get_q(h[0])
            if next_max < 0:
                qvals[h[1]] = final_value
            else:
                qvals[h[1]] = qvals[h[1]] * (
                            1.0 - self.learning_rate) + self.learning_rate * self.value_discount * next_max

            next_max = max(qvals)

    def new_game(self, side):
        self.side = side
        self.move_history = []



Define Gameing Mode

In [100]:
def play_game(board: Board, player1: Player, player2: Player):
    player1.new_game(CROSS)
    player2.new_game(NAUGHT)
    board.reset()
    
    finished = False
    while not finished:
        result, finished = player1.move(board)
        if finished:
            if result == GameResult.DRAW:
                final_result = GameResult.DRAW
            else:
                final_result =  GameResult.CROSS_WIN
        else:
            result, finished = player2.move(board)
            if finished:
                if result == GameResult.DRAW:
                    final_result =  GameResult.DRAW
                else:
                    final_result =  GameResult.NAUGHT_WIN
        
    player1.final_result(final_result)
    player2.final_result(final_result)
    return final_result

def battle(player1: Player, player2: Player, num_games):
    board = Board()
    draw_count = 0
    cross_count = 0
    naught_count = 0
    for _ in range(num_games):
        result = play_game(board, player1, player2)
        if result == GameResult.CROSS_WIN:
            cross_count += 1
        elif result == GameResult.NAUGHT_WIN:
            naught_count += 1
        else:
            draw_count += 1

    print("After {} game we have draws: {}, Player 1 wins: {}, and Player 2 wins: {}.".format(num_games, draw_count,
                                                                                         cross_count, naught_count))

    print("Which gives percentages of draws: {:.2%}, Player 1 wins: {:.2%}, and Player 2 wins:  {:.2%}".format(
        draw_count / num_games, cross_count / num_games, naught_count / num_games))

Define to evaluate the players


In [101]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
def eval_players(p1 : Player, p2 : Player, num_battles : int, games_per_battle = 100, loc='best'):
    p1_wins = []
    p2_wins = []
    draws = []
    count = []    

    for i in range(num_battles):
        p1win, p2win, draw = battle(p1, p2, games_per_battle)
        p1_wins.append(p1win*100.0/games_per_battle)
        p2_wins.append(p2win*100.0/games_per_battle)
        draws.append(draw*100.0/games_per_battle)
        count.append(i*games_per_battle)
        p1_wins.append(p1win*100.0/games_per_battle)
        p2_wins.append(p2win*100.0/games_per_battle)
        draws.append(draw*100.0/games_per_battle)
        count.append((i+1)*games_per_battle)

    plt.ylabel('Game outcomes in %')
    plt.xlabel('Game number')

    plt.plot(count, draws, 'r-', label='Draw')
    plt.plot(count, p1_wins, 'g-', label='Player 1 wins')
    plt.plot(count, p2_wins, 'b-', label='Player 2 wins')
    plt.legend(loc=loc, shadow=True, fancybox=True, framealpha =0.7)

Evalute the performance between Minimax and Simple NNQ

In [104]:
player1 = MinMaxAgent()
player2 = TQPlayer()
battle(player1,player2,10000)

After 10000 game we have draws: 9905, Player 1 wins: 95, and Player 2 wins: 0.
Which gives percentages of draws: 99.05%, Player 1 wins: 0.95%, and Player 2 wins:  0.00%


In [106]:
player1=MinMaxAgent()
player2=RndMinMaxAgent()
battle(player1,player2,1000)

After 1000 game we have draws: 1000, Player 1 wins: 0, and Player 2 wins: 0.
Which gives percentages of draws: 100.00%, Player 1 wins: 0.00%, and Player 2 wins:  0.00%


In [109]:
player1=TQPlayer()
player2=RndMinMaxAgent()
battle(player1,player2,10000)

After 10000 game we have draws: 9016, Player 1 wins: 0, and Player 2 wins: 984.
Which gives percentages of draws: 90.16%, Player 1 wins: 0.00%, and Player 2 wins:  9.84%
